In [1]:
import plistlib
import mutagen
from pathlib import Path
from urllib.parse import unquote, urlparse
import subprocess
import os

In [2]:
os.chdir("/Users/toph/Local/Projects/vibechecker-scratch")

In [3]:
def get_playlists_by_name(name):
    return [pl for pl in playlists if pl["Name"] == name]


def get_parent(pl):
    if "Parent Persistent ID" not in pl.keys():
        return None
    else:
        parent = [item for item in playlists
        if item["Playlist Persistent ID"] == pl["Parent Persistent ID"]][0]
        return parent


def get_ancestors(pl):
    this_pl = pl
    ancestors = []
    while "Parent Persistent ID" in this_pl.keys():
        this_plp = get_parent(this_pl)
        ancestors.append(this_plp)
        this_pl = this_plp
    return ancestors


def playlist_tracks(playlist):
    track_ids = [item["Track ID"] for item in playlist["Playlist Items"]]
    # return [track for i, track in itl["Tracks"].items() if track["Track ID"] in track_ids]
    playlist_tracks = []
    for track_id in track_ids:
        playlist_tracks.extend([track for i, track in itl["Tracks"].items() if track["Track ID"] == track_id])
    return playlist_tracks

In [4]:
with open(Path("/Users/toph/Music/Library.xml").absolute(), "rb") as xml_file:
    itl = plistlib.load(xml_file)


## Transform data

### Vibes

In [5]:
playlists = itl["Playlists"]

In [6]:
vibes_folder = [pl for pl in playlists if pl["Name"] == "2. Vibes"][0]

In [7]:
music_vibes = [
    pl for pl in playlists
    if vibes_folder in get_ancestors(pl)
]

In [8]:
sql_vibes = []
for pl in music_vibes:
    vibe = {}
    vibe["vibe_id"] = vibe["playlist_id"] = pl["Playlist ID"]
    vibe["name"] = pl["Name"]
    vibe["parent_id"] = get_parent(pl)["Playlist ID"]
    vibe["type"] = "label" if "Folder" not in pl.keys() else "folder"
    sql_vibes.append(vibe)

In [9]:
sql_vibes[0:6]

[{'vibe_id': 163260,
  'playlist_id': 163260,
  'name': '1. Elements – Instruments, Beats, Composition, etc.',
  'parent_id': 162450,
  'type': 'folder'},
 {'vibe_id': 163654,
  'playlist_id': 163654,
  'name': '1. Instruments, Sounds',
  'parent_id': 163260,
  'type': 'folder'},
 {'vibe_id': 163739,
  'playlist_id': 163739,
  'name': 'Acoustic Parts',
  'parent_id': 163654,
  'type': 'label'},
 {'vibe_id': 163756,
  'playlist_id': 163756,
  'name': 'Beatless, Ambient',
  'parent_id': 163654,
  'type': 'label'},
 {'vibe_id': 163775,
  'playlist_id': 163775,
  'name': 'Beats Mostly',
  'parent_id': 163654,
  'type': 'label'},
 {'vibe_id': 163784,
  'playlist_id': 163784,
  'name': 'Beats Only',
  'parent_id': 163654,
  'type': 'label'}]

### Tracks

In [10]:
def try_key(dict, key, default = None):
    return dict[key] if key in dict.keys() else default

In [11]:
sql_tracks = []

for _, music_track in itl["Tracks"].items():
    track = {}
    track["track_id"] = music_track["Track ID"]
    track["name"] = music_track["Name"]
    track["artist"] = try_key(music_track, "Artist", "")
    track["album"] = try_key(music_track, "Album", "")
    track["tracknum"] = try_key(music_track, "Track Number", 0)
    sql_tracks.append(track)


In [12]:
sql_tracks[0]

{'track_id': 15129,
 'name': 'Music on My Teeth (feat. José Gonzalez)',
 'artist': 'DJ Koze',
 'album': 'Knock Knock',
 'tracknum': 5}

### Track-Vibe Associations

In [13]:
mv_test = [p for p in playlists if p["Playlist ID"] == 169670]

In [14]:
len(music_vibes)

200

In [15]:
track_vibes = []
for playlist in music_vibes:
    if "Playlist Items" not in playlist.keys():
        continue
    vibe_tracks = []
    if playlist["Playlist ID"] == 169670:
        print("playlist")
    for playlist_item in playlist["Playlist Items"]:
        if playlist_item["Track ID"] == 16067 and playlist["Playlist ID"] == 169670:
            print("track")
        vibe_track = {}
        vibe_track["vibe_id"] = playlist["Playlist ID"]
        vibe_track["track_id"] = playlist_item["Track ID"]
        vibe_track["track_vibe_id"] = int(str(vibe_track["track_id"]) + str(vibe_track["vibe_id"]))
        vibe_track["value"] = 1.0
        vibe_tracks.append(vibe_track)
        if vibe_track["track_vibe_id"] == 16534423393:
            print("bad vibes")
    track_vibes.extend(vibe_tracks)

playlist


In [16]:
track_vibes

rack_id': 70730,
  'track_vibe_id': 70730163867,
  'value': 1.0},
 {'vibe_id': 163867,
  'track_id': 16061,
  'track_vibe_id': 16061163867,
  'value': 1.0},
 {'vibe_id': 163867,
  'track_id': 70812,
  'track_vibe_id': 70812163867,
  'value': 1.0},
 {'vibe_id': 163867,
  'track_id': 70862,
  'track_vibe_id': 70862163867,
  'value': 1.0},
 {'vibe_id': 163867,
  'track_id': 70871,
  'track_vibe_id': 70871163867,
  'value': 1.0},
 {'vibe_id': 163867,
  'track_id': 71592,
  'track_vibe_id': 71592163867,
  'value': 1.0},
 {'vibe_id': 163867,
  'track_id': 71365,
  'track_vibe_id': 71365163867,
  'value': 1.0},
 {'vibe_id': 163867,
  'track_id': 71368,
  'track_vibe_id': 71368163867,
  'value': 1.0},
 {'vibe_id': 163867,
  'track_id': 71562,
  'track_vibe_id': 71562163867,
  'value': 1.0},
 {'vibe_id': 163867,
  'track_id': 71565,
  'track_vibe_id': 71565163867,
  'value': 1.0},
 {'vibe_id': 163867,
  'track_id': 71577,
  'track_vibe_id': 71577163867,
  'value': 1.0},
 {'vibe_id': 163867,
  '

In [17]:
# There is apparently non-uniqueness in track_vibes
import collections
track_vibe_ids = [x["track_vibe_id"] for x in track_vibes]
dup_ids = [item for item, count in collections.Counter(track_vibe_ids).items() if count > 1]
dup_ids

[23393165344, 20556167526, 21360168100, 23396168866]

In [18]:
duplicated_vibes = [vibe for vibe in track_vibes if vibe["track_vibe_id"] in dup_ids]

In [19]:
duplicated_vibes

[{'vibe_id': 165344,
  'track_id': 23393,
  'track_vibe_id': 23393165344,
  'value': 1.0},
 {'vibe_id': 165344,
  'track_id': 23393,
  'track_vibe_id': 23393165344,
  'value': 1.0},
 {'vibe_id': 167526,
  'track_id': 20556,
  'track_vibe_id': 20556167526,
  'value': 1.0},
 {'vibe_id': 167526,
  'track_id': 20556,
  'track_vibe_id': 20556167526,
  'value': 1.0},
 {'vibe_id': 168100,
  'track_id': 21360,
  'track_vibe_id': 21360168100,
  'value': 1.0},
 {'vibe_id': 168100,
  'track_id': 21360,
  'track_vibe_id': 21360168100,
  'value': 1.0},
 {'vibe_id': 168866,
  'track_id': 23396,
  'track_vibe_id': 23396168866,
  'value': 1.0},
 {'vibe_id': 168866,
  'track_id': 23396,
  'track_vibe_id': 23396168866,
  'value': 1.0}]

In [20]:
dv = duplicated_vibes[0]

In [21]:
dv["track_id"]

23393

In [22]:
[p for p in playlists if p["Playlist ID"] == dv["vibe_id"]]

[{'Name': 'Energy 3',
  'Description': '',
  'Playlist ID': 165344,
  'Playlist Persistent ID': '9D540B8B771A0207',
  'Parent Persistent ID': '8DC7885852007E94',
  'All Items': True,
  'Playlist Items': [{'Track ID': 20140},
   {'Track ID': 20134},
   {'Track ID': 21307},
   {'Track ID': 21287},
   {'Track ID': 21453},
   {'Track ID': 22145},
   {'Track ID': 22106},
   {'Track ID': 22250},
   {'Track ID': 22049},
   {'Track ID': 22384},
   {'Track ID': 22856},
   {'Track ID': 21648},
   {'Track ID': 21459},
   {'Track ID': 22399},
   {'Track ID': 21465},
   {'Track ID': 20397},
   {'Track ID': 18560},
   {'Track ID': 22708},
   {'Track ID': 17072},
   {'Track ID': 23249},
   {'Track ID': 23252},
   {'Track ID': 23279},
   {'Track ID': 24300},
   {'Track ID': 19368},
   {'Track ID': 19380},
   {'Track ID': 23816},
   {'Track ID': 67828},
   {'Track ID': 67976},
   {'Track ID': 67955},
   {'Track ID': 67819},
   {'Track ID': 68451},
   {'Track ID': 23393},
   {'Track ID': 23393},
   {'Tr

In [23]:
len(duplicated_vibes)

8

## Initialize Schema

In [24]:
subprocess.run("rm -f vibechecker.db".split(" "))

CompletedProcess(args=['rm', '-f', 'vibechecker.db'], returncode=0)

In [25]:
subprocess.run("sqlite3 -init init-db.sql vibechecker.db '.exit'".split(" "))

CompletedProcess(args=['sqlite3', '-init', 'init-db.sql', 'vibechecker.db', "'.exit'"], returncode=1)

## Insert stuff into tables!

In [26]:
import sqlite3

In [27]:
conn = sqlite3.connect("vibechecker.db")

In [28]:
cursor = conn.cursor()

In [29]:
cursor.executemany("""
    INSERT INTO vibes
        (vibe_id, playlist_id, name, parent_id, type)
    VALUES
        (:vibe_id, :playlist_id, :name, :parent_id, :type)""", sql_vibes)
conn.commit()

In [30]:
track_vibes[0]

{'vibe_id': 163260,
 'track_id': 16067,
 'track_vibe_id': 16067163260,
 'value': 1.0}

In [31]:
cursor.executemany("""
    INSERT INTO tracks
        (track_id, name, artist, album, tracknum)
    VALUES
        (:track_id, :name, :artist, :album, :tracknum)""", sql_tracks)
conn.commit()

In [32]:
# The "insert or ignore" statement is to deal with duplicate entries.
cursor.executemany("""
    INSERT OR IGNORE INTO track_vibes
        (track_vibe_id, vibe_id, track_id, value)
    VALUES
        (:track_vibe_id, :vibe_id, :track_id, :value)""", track_vibes)
conn.commit()